In [ ]:
%load_ext autoreload
%autoreload 2
from covid_constants_and_util import *
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import json
import datetime
import copy
import geopandas as gpd
import dask
import helper_methods_for_aggregate_data_analysis as helper
import h5py
import re

JUST_TESTING = True

print("OKAY")

Setting numpy cores to 1
Running code on Deep; at Stanford=False
In constants.py, ALL_WEEKLY_STRINGS: ['2018-12-31', '2019-01-07', '2019-01-14', '2019-01-21', '2019-01-28', '2019-02-04', '2019-02-11', '2019-02-18', '2019-02-25', '2019-03-04', '2019-03-11', '2019-03-18', '2019-03-25', '2019-04-01', '2019-04-08', '2019-04-15', '2019-04-22', '2019-04-29', '2019-05-06', '2019-05-13', '2019-05-20', '2019-05-27', '2019-06-03', '2019-06-10', '2019-06-17', '2019-06-24', '2019-07-01', '2019-07-08', '2019-07-15', '2019-07-22', '2019-07-29', '2019-08-05', '2019-08-12', '2019-08-19', '2019-08-26', '2019-09-02', '2019-09-09', '2019-09-16', '2019-09-23', '2019-09-30', '2019-10-07', '2019-10-14', '2019-10-21', '2019-10-28', '2019-11-04', '2019-11-11', '2019-11-18', '2019-11-25', '2019-12-02', '2019-12-09', '2019-12-16', '2019-12-23', '2019-12-30', '2020-01-06', '2020-01-13', '2020-01-20', '2020-01-27', '2020-02-03', '2020-02-10', '2020-02-17', '2020-02-24', '2020-03-02', '2020-03-09', '2020-03-16', '

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [26]:
# make sure we don't append onto existing files [for H5 files]. 
assert not os.path.exists(os.path.join(helper.ANNOTATED_H5_DATA_DIR, helper.CHUNK_FILENAME))

In [ ]:
# Write out dataframe of Census data for use in subsequent analysis. 
# helper.write_out_acs_5_year_data()

In [24]:
# read in individual dataframes for monthly and weekl data [raw SafeGraph data].
dask.config.set(pool=ThreadPool(10))

print("helper.ALL_WEEKLY_STRINGS:", helper.ALL_WEEKLY_STRINGS)


all_monthly_dfs = []
all_weekly_dfs = []   

print( glob.glob(r'/media/gpu/easystore/Safegraph/Weekly Places Patterns v2 (until 2020-06-15)/main-file/*.gz'))
print("helper.ALL_WEEKLY_STRINGS:", helper.ALL_WEEKLY_STRINGS)
ALL_WEEKLY_STRINGS = sorted([os.path.basename(a).replace('-weekly-patterns.csv.gz', '') for a in glob.glob(r'/media/gpu/easystore/Safegraph/Weekly Places Patterns v2 (until 2020-06-15)/main-file/*.gz')])  # Huan

ALL_WEEKLY_STRINGS = [x for x in ALL_WEEKLY_STRINGS if x[:4] > str(2019) ]

print("helper.ALL_WEEKLY_STRINGS:", helper.ALL_WEEKLY_STRINGS)
print("ALL_WEEKLY_STRINGS:", ALL_WEEKLY_STRINGS)
# for week_string in helper.ALL_WEEKLY_STRINGS:
# for week_string in ALL_WEEKLY_STRINGS:
#     print("Process:", week_string)
#     all_weekly_dfs.append(helper.load_patterns_data(week_string=week_string, just_testing=JUST_TESTING))
    
for month, year in [
             (1, 2019),(2, 2019),(3, 2019),(4, 2019),(5, 2019),(6, 2019),(7, 2019),(8, 2019),(9, 2019),(10, 2019),(11, 2019),(12, 2019),
             (1, 2020),(2, 2020)][::-1]:
    # Note ::-1: we load most recent files first so we will take their places info if it is available.
    all_monthly_dfs.append(helper.load_patterns_data(month=month, year=year, just_testing=JUST_TESTING))
    


helper.ALL_WEEKLY_STRINGS: ['2018-12-31', '2019-01-07', '2019-01-14', '2019-01-21', '2019-01-28', '2019-02-04', '2019-02-11', '2019-02-18', '2019-02-25', '2019-03-04', '2019-03-11', '2019-03-18', '2019-03-25', '2019-04-01', '2019-04-08', '2019-04-15', '2019-04-22', '2019-04-29', '2019-05-06', '2019-05-13', '2019-05-20', '2019-05-27', '2019-06-03', '2019-06-10', '2019-06-17', '2019-06-24', '2019-07-01', '2019-07-08', '2019-07-15', '2019-07-22', '2019-07-29', '2019-08-05', '2019-08-12', '2019-08-19', '2019-08-26', '2019-09-02', '2019-09-09', '2019-09-16', '2019-09-23', '2019-09-30', '2019-10-07', '2019-10-14', '2019-10-21', '2019-10-28', '2019-11-04', '2019-11-11', '2019-11-18', '2019-11-25', '2019-12-02', '2019-12-09', '2019-12-16', '2019-12-23', '2019-12-30', '2020-01-06', '2020-01-13', '2020-01-20', '2020-01-27', '2020-02-03', '2020-02-10', '2020-02-17', '2020-02-24', '2020-03-02', '2020-03-09', '2020-03-16', '2020-03-23', '2020-03-30', '2020-04-06', '2020-04-13', '2020-04-20', '2020-

ValueError: Usecols do not match columns, columns expected but not found: ['top_category', 'longitude', 'sub_category', 'polygon_wkt', 'latitude', 'naics_code', 'polygon_class']

In [ ]:
# Merge monthly DFs into a single dataframe. Each row is one POI. 
base = all_monthly_dfs[0]
core = all_monthly_dfs[1].columns.intersection(base.columns).to_list()
for i, df in enumerate(all_monthly_dfs[1:]):
    print(i)
    # merge all new places into base so that core info is not nan for new sgids
    new_places = df.loc[df.index.difference(base.index)][core]
    base = pd.concat([base, new_places], join='outer', sort=False)
    # can now left merge in the df because all sgids will be in base
    cols_to_use = df.columns.difference(base.columns).to_list()
    base =  pd.merge(base, df[cols_to_use], left_index=True, right_index=True, how='left')


In [ ]:
# Merge in weekly dataframes. Just merge on SafeGraph ID, left merge. 
# This means that our final POI set is those that have both monthly and weekly data. 
# at the end of this cell we will have a single dataframe. 

for i, weekly_df in enumerate(all_weekly_dfs):
    print("\n\n********Weekly dataframe %i/%i" % (i + 1, len(all_weekly_dfs)))
    assert len(base.columns.intersection(weekly_df.columns)) == 0
    
    ids_in_weekly_but_not_monthly = set(weekly_df.index) - set(base.index)
    print("Warning: %i/%i POIs in weekly but not monthly data; dropping these" % (len(ids_in_weekly_but_not_monthly), 
                                                                  len(df)))
    base = pd.merge(base, weekly_df, how='left', left_index=True, right_index=True, validate='one_to_one')
    print("Missing data for weekly columns")
    print(pd.isnull(base[weekly_df.columns]).mean())

In [ ]:
# sanity check: how much do weekly visits change if we drop parent IDs. 
parent_ids = set(base['parent_safegraph_place_id'].dropna())
first_week_of_march_cols = ['hourly_visits_2020.3.%i.%i' % (i, j) for i in range(1, 8) for j in range(24)]
total_daily_child_visits = base.loc[~pd.isnull(base['parent_safegraph_place_id']), first_week_of_march_cols].dropna().values.sum()
total_daily_parent_visits = base.loc[base.index.map(lambda x:x in parent_ids), first_week_of_march_cols].dropna().values.sum()
total_daily_visits = base[first_week_of_march_cols].dropna().values.sum()
print("Total daily child visits are fraction %2.3f of total visits; parent visits are %2.3f; dropping parent visits" 
      % (total_daily_child_visits/total_daily_visits, total_daily_parent_visits/total_daily_visits))

# Drop parents to avoid double-counting visits. 
base = base.loc[base.index.map(lambda x:x not in parent_ids)]

In [ ]:
# annotate with demographic info and save dataframe. Dataframe is saved in h5py format, separated into chunks. 

annotated = base.sample(frac=1) # shuffle so rows are in random order [in case we want to prototype on subset].
annotated = helper.annotate_with_demographic_info_and_write_out_in_chunks(annotated, just_testing=JUST_TESTING)

In [ ]:
# Stratify by MSA and write out outfiles.  
just_in_msas = annotated.loc[annotated['poi_lat_lon_Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area']
assert pd.isnull(just_in_msas['poi_lat_lon_CBSA Title']).sum() == 0  # POIs in MSAs must have CBSA title
print("%i/%i POIs are in MSAs (%i MSAs total)" % (len(just_in_msas), 
                                                  len(annotated), 
                                                  len(set(just_in_msas['poi_lat_lon_CBSA Title']))))
grouped_by_msa = just_in_msas.groupby('poi_lat_lon_CBSA Title')
total_written_out = 0
for msa_name, small_d in grouped_by_msa:
    small_d = small_d.copy().sample(frac=1) # make sure rows in random order. 
    small_d.index = range(len(small_d))
    name_without_spaces = re.sub('[^0-9a-zA-Z]+', '_', msa_name)
    filename = os.path.join(helper.STRATIFIED_BY_AREA_DIR, '%s.csv' % name_without_spaces)
    for k in ['aggregated_cbg_population_adjusted_visitor_home_cbgs', 'aggregated_visitor_home_cbgs']:
        small_d[k] = small_d[k].map(lambda x:json.dumps(dict(x))) # cast to json so properly saved in CSV. 
    small_d.to_csv(filename)
    print("Wrote out dataframe with %i POIs to %s" % (len(small_d), '%s.csv' % name_without_spaces))
    total_written_out += 1
print("Total written out: %i" % total_written_out)